# ARIMA SARIMA 가이드라인

## EDA 사전 세팅

In [2]:
# EDA 사전 세팅
%matplotlib inline

import time
import os
import openai
import warnings

os.environ['KMP_WARNINGS'] = 'off'
warnings.filterwarnings('ignore')

from functools import reduce
import matplotlib as mpl
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
import seaborn as sns  #데이터 시각화 
sns.set(font="AppleGothic",

        rc={"axes.unicode_minus":False},
        
        style='darkgrid')

In [ ]:
#ARIMA package
!pip install pmdarima # <- ARIMA 파키지 설치하는 코드 만약 ARIMA가 불러와 지지 않는다면 해당 코드를 실행해 보자 

In [ ]:
# 모델링 시 필요한 리이브러리
from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA # <- ARIMA 모델링
from statsmodels.tsa.statespace.sarimax import SARIMAX # <- SARIMA 모델링
import statsmodels.api as sm # <- 시계열 분해
import itertools # <- ARIMA 와 SARIMA 모델 최적 파라미터(p, d, q) 찾아주는 코드 작성할 떄 필요 

## 데이터 불러오기

In [ ]:
# 데이터를 불러오시오

## 데이터 전처리

가이드라인 <br> 
1. 원본 데이터의 일시 변수를 to_datetime 처리하여 시꼐열 데이터 만들기
2. datetime 처리한 일시 변수를 인덱스로 지정하기(set_index)
3. 인덱스(일시) 기준으로 데이터 정렬하기
4. 각 구에서 대여량이 0인 날짜 찾기
5. 0인 날짜 행 drop하기
6. 보간법을 통해 데이터 채워 넣기

위 가이드 라인에서 어렵다고 해당되는 부분은 보간법이 유일함으로 보간법에 대한 코드만 간략히 제공<br>
보간법이 뭔지는 각각 찾아볼 것

HINT: 데이터의 시간 간격을 일별로 설정하고 결측치(데이터가 0)를 직전의 값으로 채워넣어라.

In [ ]:
data.asfreq(freq= '?', method= '?') # 파라미터는 찾아보시오

## 시계열 분해

가이드 라인
1. 시계열을 [30,60,72,90,360] 일 간격으로 분해하여 그래프를 그리시오<br>
시계열 분해 코드 베이스 라인은 아래와 같다.<br>
아래 파라미터에서 x에는 데이터를, model은 가법 모델을 사용하고 기간은 가이드 라인을 참고하라

In [ ]:
decomposition = sm.tsa.seasonal_decompose(x='?',model='?',period='?')

fig = decomposition.plot()
fig.set_size_inches(11,9)
plt.title('model = %s / period = %s' %(model,period),pad=480)
plt.show()

## ACF 및 PACF 확인

아래 코드는 자기상관함수와 편자기상관함수의 개형을 확인하는 코드이다. <br>
파라미터를 채워넣어 시차 50, 150 기준으로 각 구별 ACF 및 PACF의 절단점을 확인해라. 

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(15,5))
sm.graphics.tsa.plot_acf(x='?',lags= '?' ,ax=ax[0])
sm.graphics.tsa.plot_pacf(x='?',lags='?',ax=ax[1])

## 데이터 분할

train_test_split 메서드를 활용하여 데이터를 분리하여라<br> test_size = 0.2

## log & diff

train 데이터와 test 데이터에 각각 로그 및 차분을 실시하시오 <br>
아래는 로그 및 차분을 수행하는 가장 기본적인 베이스라인 코드이다. 기왕이면 함수화하면 좋다.

In [ ]:
log_data = np.log1p(data)
diff_data = log_data.diff()

### 로그 및 차분을 진행한 데이터의 각 구별 ACF PACF 그래프를 그려라 시차는 10으로 한다. 

In [ ]:
# 그래프를 그리는 코드는 위에 존재

## ML 모델링

### ARIMA 모델 파라미터 찾기

아래는 p,d,q ARIMA 모델의 최적 파라미터 찾는 함수이다. 해당 함수를 이용해서 최적 파라미터를 찾아라

In [ ]:
def find_pdq(data,col):
  p = range(0,2)
  d = range(0,2)
  q = range(0,3)

  pdq = list(itertools.product(p,d,q))
  aic = []

  for i in pdq:
    model = sm.tsa.arima.ARIMA(data[col], order=i)
    model_fit = model.fit()#transparams=False)
    aic.append(model_fit.aic)

  optimal = [(pdq[i],j) for i,j in enumerate(aic) if j == min(aic)]
  return optimal, aic

In [ ]:
pdq = find_pdq('?','?')[0][0]

### ARIMA

아래는 모델링을 통해 예측값을 도츨하는 함수와 차분 킻 로그 변환한 데ㅣ터를 원래 데이터로 변환해주는 함수이다.

In [ ]:
def modeling_arima(modeling_data, fore_data, col):
  model_arima = sm.tsa.arima.ARIMA(modeling_data[col],order = pdq[0])
  history = model_arima.fit()
  history.summary()
  pred = history.get_forecast(len(fore_data[col]))
  return pred

def rev_diff(train_last,pred, col):
  pred_values = pred.predicted_mean
  pred_cum = train_last+pred_values.cumsum()
  pred_exp_cum = np.expm1(pred_cum)
  return pred_exp_cum

위 함수를 이용하여 각 구별 예측값을 도출하라. 베이스라인 코드는 아래와 같다.

In [ ]:
# 광진구의 예측값
col = '광진구'
pred = modeling_arima('?', '?', '?')
train_last = log_diff('?')[0][col][-1]
rev_pred = rev_diff('?','?',col)
rev_pred

In [ ]:
plt.plot(rev_pred) 예측값 그래프

### MAE 도출

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(ml_test[col][1:], rev_pred)

1. 각 구별 예측값을 submission 파일을 불러와 하나의 데이터 프레임으로 만들어라
2. 데이콘에 제출해보자
